In [1]:
import warnings
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')


def carregar_arquivo():
    caminho = filedialog.askopenfilename(
        title="Selecione o arquivo de atendimentos",
        filetypes=[("Arquivos Excel", "*.xlsx")]
    )
    if not caminho:
        return None
    try:
        return pd.read_excel(caminho)
    except Exception as e:
        messagebox.showerror("Erro ao carregar planilha", str(e))
        return None


def verificar_colunas(df, colunas):
    ausentes = [col for col in colunas if col not in df.columns]
    if ausentes:
        messagebox.showerror("Colunas ausentes", f"As colunas seguintes não foram encontradas na planilha:\n{', '.join(ausentes)}")
        return False
    return True


def calcular_estatisticas(df):
    colunas = ["Usuário Conclusão", "Motivos Conclusão/Retorno Analista", "Mantido Fornecimento"]
    if not verificar_colunas(df, colunas):
        return None, 0, 0, 0, 0, 0

    df_filtrado = df[colunas]
    motivos_desejados = {
        "Acordo de mão de obra", "Alteração de loja", "Argumentação",
        "Encomenda", "Item sem giro", "Item fora de cesta"
    }

    df_filtrado = df_filtrado[df_filtrado["Motivos Conclusão/Retorno Analista"].isin(motivos_desejados)]
    df_grouped = df_filtrado.groupby("Usuário Conclusão")["Mantido Fornecimento"].value_counts().unstack(fill_value=0)

    df_grouped["Total Atendimentos"] = df_grouped.get("SIM", 0) + df_grouped.get("NAO", 0)
    df_grouped["Retenção (%)"] = (df_grouped.get("SIM", 0) / df_grouped["Total Atendimentos"]).fillna(0).round(2) * 100

    total_sim = df_grouped.get("SIM", 0).sum()
    total_atend = df_grouped["Total Atendimentos"].sum()
    retencao_equipe = (total_sim / total_atend * 100) if total_atend else 0

    grupo_a = ["VICTOR MACHADO MARTINS", "LUISA BASTOS DE MOURA"]
    grupo_b = ["FELIPE JARDIM NOVAES", "YGOR PENA NEGRELLI"]

    def calcular_grupo(nomes):
        presentes = [nome for nome in nomes if nome in df_grouped.index]
        mantidos = sum(df_grouped.loc[nome]["SIM"] for nome in presentes) if presentes else 0
        total = sum(df_grouped.loc[nome]["Total Atendimentos"] for nome in presentes) if presentes else 0
        retencao = (mantidos / total * 100) if total else 0
        return mantidos, total, retencao

    _, total_a, retencao_a = calcular_grupo(grupo_a)
    _, total_b, retencao_b = calcular_grupo(grupo_b)

    df_grouped.sort_values(by=["Total Atendimentos", "Retenção (%)"], ascending=[False, False], inplace=True)

    return df_grouped, retencao_equipe, retencao_a, retencao_b, total_a, total_b


def atualizar_resultado(df_resultado, retencao_equipe, ret_a, ret_b, total_a, total_b):
    treeview.delete(*treeview.get_children())

    def inserir(linha, total, retencao):
        treeview.insert("", "end", values=(linha, int(total), f"{retencao:.2f}%"))

    inserir("EQUIPE", df_resultado["Total Atendimentos"].sum(), retencao_equipe)
    inserir("BLOCO A", total_a, ret_a)
    inserir("BLOCO B", total_b, ret_b)

    for usuario, row in df_resultado.iterrows():
        inserir(usuario, row["Total Atendimentos"], row["Retenção (%)"])


def processar_tratados():
    df = carregar_arquivo()
    if df is None:
        return

    resultados = calcular_estatisticas(df)
    if resultados[0] is not None:
        atualizar_resultado(*resultados)


# Interface gráfica
root = tk.Tk()
root.title("Painel de Atendimentos Tratados")
root.geometry("720x540")
root.resizable(False, False)
root.configure(bg="#ffffff")

# Estilo personalizado
style = ttk.Style()
style.theme_use("clam")

# Cabeçalhos
style.configure("Treeview.Heading",
                font=("Segoe UI", 10, "bold"),
                background="#003366",
                foreground="white")

# Corpo da tabela
style.configure("Treeview",
                font=("Segoe UI", 10),
                background="white",
                fieldbackground="white",
                foreground="#003366",
                rowheight=28)

style.map("Treeview",
          background=[("selected", "#cce6ff")],
          foreground=[("selected", "black")])

# Frame
frame = ttk.Frame(root, padding=20)
frame.pack(fill="both", expand=True)

# Botão
btn = ttk.Button(frame, text="📊 Processar Dados de Tratados", command=processar_tratados)
btn.pack(pady=10)

# Treeview
columns = ("USUÁRIO CONCLUSÃO", "TOTAL DE ATENDIMENTOS", "RETENÇÃO")
treeview = ttk.Treeview(frame, columns=columns, show="headings", height=15)

for col in columns:
    treeview.heading(col, text=col)
    treeview.column(col, anchor="center", width=220)

treeview.pack(pady=10, fill="both", expand=True)

# Scrollbar
scrollbar = ttk.Scrollbar(frame, orient="vertical", command=treeview.yview)
treeview.configure(yscroll=scrollbar.set)
scrollbar.pack(side="right", fill="y")

root.mainloop()